In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

# Load your dataset
df = pd.read_csv('sfmerged.csv')

# Function to calculate Euclidean distance between players
def calculate_distance(p1_x, p1_y, p2_x, p2_y):
    return np.sqrt((p1_x - p2_x)**2 + (p1_y - p2_y)**2)

# Add distance as a new column to the dataframe
df['distance'] = df.apply(lambda row: calculate_distance(row['p1_x'], row['p1_y'], row['p2_x'], row['p2_y']), axis=1)

# Add relative health and relative position columns
df['relative_health'] = df['p1_health'] - df['p2_health']
df['relative_position'] = np.where(df['distance'] < 100, 1, 0)  # 1 if close, 0 if far

# Define features and labels
features = [
    'timer', 'p1_id', 'p1_health', 'p1_x', 'p1_y', 'p1_jumping', 'p1_crouching', 'p1_in_move', 'p1_move_id',
    'p2_id', 'p2_health', 'p2_x', 'p2_y', 'p2_jumping', 'p2_crouching', 'p2_in_move', 'p2_move_id', 'distance',
    'relative_health', 'relative_position'
]
labels = [
    'p1_btn_up', 'p1_btn_down', 'p1_btn_right', 'p1_btn_left', 'p1_btn_Y', 'p1_btn_B', 'p1_btn_X', 'p1_btn_A', 'p1_btn_L', 'p1_btn_R'
]

# Convert boolean columns to integers
bool_columns = ['p1_jumping', 'p1_crouching', 'p1_in_move', 'p2_jumping', 'p2_crouching', 'p2_in_move']
df[bool_columns] = df[bool_columns].astype(int)

# Split data into training and testing sets
X = df[features]
y = df[labels]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define model architecture
model = models.Sequential([
    layers.Input(shape=(len(features),)),
    layers.Dense(256, activation='relu'),  # Increased the number of neurons in the first hidden layer
    layers.Dropout(0.3),  # Added dropout for regularization
    layers.Dense(128, activation='relu'),  # Second hidden layer
    layers.Dense(64, activation='relu'),   # Third hidden layer
    layers.Dense(len(labels), activation='sigmoid')  # For multi-label classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Save the trained model
model.save('optimized_model_with_distance.h5')
print("Optimized model saved to 'optimized_model_with_distance.h5'")


Epoch 1/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1346 - loss: 0.9237 - val_accuracy: 0.1780 - val_loss: 0.3346
Epoch 2/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1642 - loss: 0.3404 - val_accuracy: 0.1780 - val_loss: 0.3361
Epoch 3/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1741 - loss: 0.3392 - val_accuracy: 0.1780 - val_loss: 0.3318
Epoch 4/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1825 - loss: 0.3348 - val_accuracy: 0.1780 - val_loss: 0.3311
Epoch 5/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1923 - loss: 0.3335 - val_accuracy: 0.1780 - val_loss: 0.3301
Epoch 6/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1900 - loss: 0.3347 - val_accuracy: 0.1780 - val_loss: 0.3304
Epoch 7/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1870 - loss: 0.3337 - val_accuracy: 0.1780 - val_loss: 0.3300
Epoch 8/100
563/563 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1951 - loss: 0.3319 - val_accu

Optimized model saved to 'optimized_model_with_distance.h5'
